# PhysioNet/Computing in Cardiology Challenge 2020
## Classification of 12-lead ECGs
### 2. Create Training Dataset

# Setup Noteboook

In [1]:
# Import 3rd party libraries
import os
import sys
import json
import random
import pandas as pd
from sklearn.model_selection import train_test_split

# Import local Libraries
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))))
from kardioml import DATA_PATH

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Split Physionet 2020 Training Data
## Create Training Lookup File
#### Tranche 1

In [12]:
# Get meta data files
filenames = [file.split('.')[0] for file in os.listdir(os.path.join(DATA_PATH, 'physionet_2020_1', 'formatted')) 
             if 'json' in file]

# Get label for each file
data_1 = {'filename': [], 'labels': []}
for filename in filenames:
    meta_data = json.load(open(os.path.join(DATA_PATH, 'physionet_2020_1', 'formatted', '{}.json'.format(filename))))
    data_1['filename'].append(filename)
    data_1['labels'].append('Normal' if meta_data['labels'] and 'Normal' in meta_data['labels'] else 'Other')

# Combine
data_1 = pd.DataFrame(data_1)

# Add path
data_1['path'] = data_1['filename'].map(lambda filename: os.path.join(DATA_PATH, 'physionet_2020_1', 'formatted', filename))

# View DataFrame
data_1.head()

,filename,labels,path
0,A0001,Other,C:\Users\sebastian goodfellow\Documents\code\p...
1,A0002,Normal,C:\Users\sebastian goodfellow\Documents\code\p...
2,A0003,Other,C:\Users\sebastian goodfellow\Documents\code\p...
3,A0004,Other,C:\Users\sebastian goodfellow\Documents\code\p...
4,A0005,Other,C:\Users\sebastian goodfellow\Documents\code\p...


#### Tranche 2

In [13]:
# Get meta data files
filenames = [file.split('.')[0] for file in os.listdir(os.path.join(DATA_PATH, 'physionet_2020_2', 'formatted')) 
             if 'json' in file]

# Get label for each file
data_2 = {'filename': [], 'labels': []}
for filename in filenames:
    meta_data = json.load(open(os.path.join(DATA_PATH, 'physionet_2020_2', 'formatted', '{}.json'.format(filename))))
    data_2['filename'].append(filename)
    data_2['labels'].append('Normal' if meta_data['labels'] and 'Normal' in meta_data['labels'] else 'Other')

# Combine
data_2 = pd.DataFrame(data_2)

# Add path
data_2['path'] = data_2['filename'].map(lambda filename: os.path.join(DATA_PATH, 'physionet_2020_2', 'formatted', filename))

# View DataFrame
data_2.head()

,filename,labels,path
0,Q0001,Other,C:\Users\sebastian goodfellow\Documents\code\p...
1,Q0002,Other,C:\Users\sebastian goodfellow\Documents\code\p...
2,Q0003,Other,C:\Users\sebastian goodfellow\Documents\code\p...
3,Q0004,Other,C:\Users\sebastian goodfellow\Documents\code\p...
4,Q0005,Other,C:\Users\sebastian goodfellow\Documents\code\p...


#### Combine Tranche 1 and 2

In [15]:
# Merge
data = pd.concat([data_1, data_2], ignore_index=True, axis=0).reset_index(drop=True)

# View DataFrame
data.head()

,filename,labels,path
0,A0001,Other,C:\Users\sebastian goodfellow\Documents\code\p...
1,A0002,Normal,C:\Users\sebastian goodfellow\Documents\code\p...
2,A0003,Other,C:\Users\sebastian goodfellow\Documents\code\p...
3,A0004,Other,C:\Users\sebastian goodfellow\Documents\code\p...
4,A0005,Other,C:\Users\sebastian goodfellow\Documents\code\p...


In [18]:
data[data['labels'] == 'Normal'].shape

(922, 3)

In [26]:
data[data['labels'] == 'Other'].shape

(9408, 3)

In [19]:
data.shape

(10330, 3)

In [21]:
1/ (922/10330)

11.203904555314534

In [24]:
11.203904555314534/11.203904555314534

1.0

In [25]:
1/ ((10330-922)/10330) / 11.203904555314534

0.0980017006802721

## Split Data 

In [16]:
# Set split ratios
train_size = 0.8
val_size = 0.2
assert train_size + val_size == 1

# Split dataset into train/evaluate
data_train, data_val = train_test_split(data, test_size=val_size, stratify=data['labels'], random_state=0, shuffle=True)

# Lookup file
training_lookup = {'train': data_train['path'].tolist(), 'val': data_val['path'].tolist()}

# Add Augmentation Datasets
## Zheng 2020

In [27]:
# Get file paths
paths = [os.path.join(DATA_PATH, 'zheng_2020', 'formatted', file.split('.')[0]) 
         for file in os.listdir(os.path.join(DATA_PATH, 'zheng_2020', 'formatted')) if 'json' in file]

# Add to lookup dictionary
# training_lookup['train'].extend(paths)
# random.shuffle(training_lookup['train'])
# random.shuffle(training_lookup['train'])
# random.shuffle(training_lookup['train'])
# random.shuffle(training_lookup['train'])
# random.shuffle(training_lookup['train'])

# Save Lookup JSON

In [17]:
# Save file
os.makedirs(os.path.join(DATA_PATH, 'deepecg_binary'), exist_ok=True)
with open(os.path.join(DATA_PATH, 'deepecg_binary', 'training_lookup.json'), 'w') as file:
    json.dump(training_lookup, file, sort_keys=True)